In [1]:
import ee
import pandas as pd
import geemap

In [2]:
# Initialize the Earth Engine API
ee.Initialize()

# Load surf spots data
file_path = "Surf Spots.csv"  # Path to your CSV file
surf_spots = pd.read_csv(file_path)

In [3]:
# Initialize the map (even though we're not displaying it here)
Map = geemap.Map()

In [4]:
# SST dataset (MODIS Aqua SST as an example)
sst_dataset = ee.ImageCollection("NOAA/CDR/OISST/V2_1") \
    .filterDate('2020-01-01', '2020-12-31')  # Change dates as needed

In [5]:
# Get the size of the dataset
dataset_size = sst_dataset.size().getInfo()

print(f"Number of images in the dataset: {dataset_size}")

Number of images in the dataset: 365


In [6]:
def get_monthly_sst_data(lat, lon, year, month, buffer_size=1000):
    print(f"--- Debugging: Month: {month}, Year: {year}, Coordinates: ({lat}, {lon}) ---")
    
    try:
        # Step 1: Create a point and buffer
        point = ee.Geometry.Point([lon, lat])
        buffer = point.buffer(buffer_size).bounds()
        print(f"Created point and buffer for coordinates.")

        # Step 2: Filter SST dataset for the specific month
        start_date = ee.Date.fromYMD(year, month, 1)
        end_date = start_date.advance(1, 'month')
        sst_image = sst_dataset.filterDate(start_date, end_date)
        
        # Debug: Print dataset size for the filtered month
        month_size = sst_image.size().getInfo()
        print(f"SST dataset size for {year}-{month:02d}: {month_size}")

        # If no data is found, return None
        if month_size == 0:
            print(f"No SST data available for {year}-{month:02d}.")
            return None
        
        # Step 3: Calculate mean SST
        sst_image = sst_image.mean()
        print("Computed mean SST for the month.")

        # Debug: Check if SST image has a valid band
        if 'sst' not in sst_image.bandNames().getInfo():
            print("No 'sst' band found in the image.")
            return None
        
        # Mask invalid SST values
        def mask_invalid(image):
            mask = image.select('sst').mask()
            return image.updateMask(mask)
        
        # Debug: Print if masking is applied
        sst = mask_invalid(sst_image)
        print("Applied masking to SST image.")

        # Step 4: Reduce region to get mean SST value
        mean_sst = sst.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=buffer,
            scale=1000,
            bestEffort=True
        ).get('sst')
        
        # Debug: Check the result of reduceRegion
        if mean_sst is None:
            print(f"No valid SST value for {year}-{month:02d} at location ({lat}, {lon}).")
            return None
        
        # Final SST value
        result = mean_sst.getInfo()
        print(f"Mean SST for {year}-{month:02d}: {result}")
        return result

    except Exception as e:
        print(f"Error in get_monthly_sst_data for {year}-{month:02d}, Coordinates: ({lat}, {lon}). Error: {e}")
        return None

In [7]:
# Function to generate valid dates for each month
def get_valid_dates(year, month):
    # Generate valid dates for the given month
    dates = []
    days_in_month = pd.to_datetime(f'{year}-{month:02d}-01').days_in_month
    for day in range(1, days_in_month + 1):
        date_str = f'{year}-{month:02d}-{day:02d}'
        dates.append(date_str)
    return dates

In [8]:
# Date range for iteration (using geemap's date handling)
start_date = ee.Date('2020-01-01')
end_date = ee.Date('2020-12-31')

In [9]:
# Iterate over each surf spot and each month in 2023 to get monthly mean SST
results = []
for index, row in surf_spots.iterrows():
    for month in range(1, 13):  # Iterate over the 12 months of 2023
        try:
            # Get monthly mean SST for the specific surf spot
            monthly_sst = get_monthly_sst_data(row['Latitude'], row['Longitude'], 2020, month)
            if monthly_sst is not None:
                # Add data for each valid day of the month
                valid_dates = get_valid_dates(2020, month)
                for date_str in valid_dates:
                    results.append({
                        'Surf Spot': row['Name'],
                        'Latitude': row['Latitude'],
                        'Longitude': row['Longitude'],
                        'Date': date_str,
                        'SST (°C)': monthly_sst
                    })
        except Exception as e:
            print(f"Error processing {row['Name']} for month {month}: {e}")
            results.append({
                'Surf Spot': row['Name'],
                'Latitude': row['Latitude'],
                'Longitude': row['Longitude'],
                'Date': f"2020-{month:02d}",
                'SST (°C)': None
            })

--- Debugging: Month: 1, Year: 2020, Coordinates: (6.84073903, 81.83699875) ---
Created point and buffer for coordinates.
SST dataset size for 2020-01: 31
Computed mean SST for the month.
Applied masking to SST image.
Mean SST for 2020-01: 2814.1935483870966
--- Debugging: Month: 2, Year: 2020, Coordinates: (6.84073903, 81.83699875) ---
Created point and buffer for coordinates.
SST dataset size for 2020-02: 29
Computed mean SST for the month.
Applied masking to SST image.
Mean SST for 2020-02: 2830.448275862069
--- Debugging: Month: 3, Year: 2020, Coordinates: (6.84073903, 81.83699875) ---
Created point and buffer for coordinates.
SST dataset size for 2020-03: 31
Computed mean SST for the month.
Applied masking to SST image.
Mean SST for 2020-03: 2890.548387096774
--- Debugging: Month: 4, Year: 2020, Coordinates: (6.84073903, 81.83699875) ---
Created point and buffer for coordinates.
SST dataset size for 2020-04: 30
Computed mean SST for the month.
Applied masking to SST image.
Mean SS

In [10]:
# Create a list for the final monthly results
monthly_results = []

# Iterate through the existing `results` list
for row in results:
    # Extract the necessary information
    surf_spot = row['Surf Spot']
    month = row['Date'][:7]  # Extract "YYYY-MM" from the Date
    sst = row['SST (°C)']
    
    # Check if this Surf Spot and Month already exists in monthly_results
    existing_entry = next((entry for entry in monthly_results if entry['Surf Spot'] == surf_spot and entry['Month'] == month), None)
    
    if existing_entry:
        # If the entry exists, skip adding (avoid duplicates)
        continue
    else:
        # Add a new entry for this Surf Spot and Month
        monthly_results.append({
            'Surf Spot': surf_spot,
            'Month': month,
            'SST (°C)': sst  # SST is already monthly mean in your results
        })

# Convert the monthly results into a DataFrame
monthly_df = pd.DataFrame(monthly_results)

# Save the DataFrame to a CSV file
output_file_path = "SurfSpot_SST(Monthly Mean 2020).csv"  # Path to save your output CSV
monthly_df.to_csv(output_file_path, index=False)

print(f"Monthly data successfully saved to {output_file_path}")

Monthly data successfully saved to SurfSpot_SST(Monthly Mean 2020).csv
